In [5]:
import numpy as np
set([1,2])

{1, 2}

In [7]:
a = set([1,2,3])
b=set([1])

In [10]:
x = a.difference(b)

In [12]:
len(set())

0

In [3]:
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy
from pprint import pprint
#import warnings

%pprint

Pretty printing has been turned OFF


In [26]:
%pprint

Pretty printing has been turned ON


In [4]:
tree = ET.parse('../Składnica-frazowa-171220/NKJP_1M_0401000002/morph_2-p/morph_2.52-s.xml')
tree

In [5]:
tree = ET.parse('../Składnica-frazowa-171220/NKJP_1M_0401000002/morph_4-p/morph_4.67-s.xml')
tree

In [151]:
tree = ET.parse('../Składnica-frazowa-171220/NKJP_1M_2002000170/morph_7-p/morph_7.78-s.xml')
tree

In [6]:
def _check_sentence(xml_tree, accept_tags=["forest","tree"]):
    
    """
    Funkcja sprawdza poprawnosc wypowiedzenia i arumentu: 
    - czy istnieje dla niego poprawne drzewo - wypowiedzenie jest poprawne jesli base_answer na polu "type" ma wartosc "FULL".
    - arumentem powinno byc drzewo o tagu korzenia rownym "forest" lub "tree".
    [W oryginalnych plikach z lasami jest to "forest", natomiast gdy z lasu tworzone sa pojedyncze drzewa,
    to maja one tag "tree"]
    
    xml_tree - las drzew lub drzewo [xml.etree.ElementTree.ElementTree]
    """
    
    if type(xml_tree) != ET.ElementTree:
        raise AssertionError("Bad argument type")
    
    
    if type(accept_tags) == str:
        accept_tags = [accept_tags]
    
    
    if not xml_tree.getroot().tag in accept_tags:
        raise AssertionError('Argument in not in [' + ",".join(accept_tags) + '] - it has tag "' + xml_tree.getroot().tag + '"' )
    
    
    base_answer_type = xml_tree.getroot().find('.//answer-data//base-answer').attrib["type"]
    correct = base_answer_type == "FULL"

    if not correct:
        raise AssertionError("Sentence is not correct: Node <base-answer> has type value " + base_answer_type  + " instead of 'FULL'")
        
    pass


def get_random_tree(forest, random_state=None):
    
    """
    Funkcja zwraca losowe drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie komunikatu
    _check_sentence(forest,"forest")
    
    # ustawiamy ziarno
    if random_state is not None:
        np.random.seed(random_state)
            
            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
    
    # definiujemy wezel ze statystykami drzewa
    # robimy to w tym iejscu zeby zachowac logiczna kolejnosc wezlow - zeby wypisywalo sie to na poczatku
    # wartosci nadamy nizej
    ET.SubElement(root_new, "tree-stats", {"height":"0","nodes":"0"})
            
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    def add_random_children(current_node_old):
        
        current_node_new = ET.SubElement(root_new, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        children_old = current_node_old.findall("children")
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return None
        random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        random_children_new = ET.SubElement(current_node_new, random_children_old.tag, random_children_old.attrib)
        for child_old in random_children_old.getchildren():
            x = ET.SubElement(random_children_new, child_old.tag, child_old.attrib)
            next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
            add_random_children(next_node)
        
    
        # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    # TODO: upewnic sie czy to jest poprawne podejscie - czy moze byc inny wezel poczatkowym
    node_0 = root_old.find('.//node[@nid="0"]') 
    
    # konstruujemy drzewo:
    add_random_children(node_0)
    
    new_tree = ET.ElementTree(root_new)
    
    th = _tree_height(new_tree, node_id=0)
    
    root_new.find("tree-stats").attrib["height"] = str(th)
    root_new.find("tree-stats").attrib["nodes"] = str(len(root_new.findall("node")))
    
    return new_tree
       
 

def number_of_trees_in_forest(forest):

    """
    Funkcja zwraca liczbe drzew w lesie forest.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """
    
    _check_sentence(forest,"forest")
    
    return int(forest.find("stats").attrib["trees"])
    
    
def get_random_negative_tree(forest, random_state=None):
    
    """
    Funkcja zwraca losowe negatywne (niepoprawne) drzewo z lasu forest.
    
    Gdy las sklada sie tylko z jednego drzewa (poprawnego) to zwracana jest wartosc None.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """
    
    _check_sentence(forest,"forest")
    
    
    n = number_of_trees_in_forest(forest)
    
    if n == 1:
        Warning("There is only one tree in the forest")
        return None
    
    else:
        while True:
            tree = get_random_tree(forest,random_state)
            if not is_positive(tree):
                return tree
    
    

In [152]:
random_tree = get_random_tree(tree)

In [156]:
ET.dump(random_tree)

<tree grammar_no="1505562921" sent_id="NKJP_1M_2002000170/morph_7-p/morph_7.78-s"><text>- Ani trochę.</text>
  <startnode from="0" to="4">wypowiedzenie</startnode>
  <forest-stats cputime="0.007831746" inferences="12165" nodes="17" trees="3" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="agataw">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="annaa">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <tree-stats height="6" nodes="13" /><node chosen="true" from="0" nid="0" subtrees="3" to="4"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w2"><child from="0" head="false" nid="1" to="1" /><child from="1" head="true" nid="15" to="3" /><child from="3" head="false" nid="11" to="4" /></children></node><node chosen="true" 

In [31]:
def terminals(tree):

    terminal_nodes = [x for x in tree.findall("node[terminal]")]

    terminals = [[(x.attrib["nid"],
                   x.find("terminal//orth").text.replace(" ", ""),  # zdarzaja sie przypadki ze token jest ze spacja i potem wyglada to jakby bylo wiecej tokenow i sie dlugosc nie zgadza
                   x.find("terminal//base").text, 
                   x.find("terminal//f").text)]  for x in terminal_nodes]

    ids = [x[0][0] for x in terminals]

    return terminals, ids 

In [153]:
terminal_nodes, ids = terminals(random_tree)
terminal_nodes

[[('2', '-', '-', 'interp')],
 [('6', 'Ani', 'ani', 'qub')],
 [('10', 'trochę', 'trochę', 'adv')],
 [('12', '.', '.', 'interp')]]

In [162]:
def get_head(tree, node_id):
    """
    Funkcja zwraca słowo, które byłoby korzeniem poddrzewa drzewa 'tree' wychodzącego z wierzchołka 'node_id' w rozbiorze zależnościowym.
    """
    
    node = tree.find(".//node[@nid='" + str(node_id) + "']")

    while not ( len(node.getchildren())==1 and node.getchildren()[0].tag=="terminal"):
        print(node, node.attrib, node.getchildren())
        try:
            head_child_id = node.find("children/child[@head='true']").attrib["nid"]
            node = tree.find(".//node[@nid='" + str(head_child_id) + "']")
        except:
            return("__head_unknown__")
        
    head = node.find("terminal//orth").text.replace(" ", "")
    
    return(head)

In [163]:
random_tree = get_random_tree(tree)
for nid in [x.attrib["nid"] for x in random_tree.findall("node")]:
    print(nid)
    print(get_head(random_tree,nid))

0
<Element 'node' at 0x7f14d820bef8> {'nid': '0', 'from': '0', 'to': '4', 'subtrees': '3', 'chosen': 'true'} [<Element 'nonterminal' at 0x7f14d8344228>, <Element 'children' at 0x7f14d820bd18>]
<Element 'node' at 0x7f14d820bc28> {'nid': '3', 'from': '1', 'to': '3', 'subtrees': '1', 'chosen': 'false'} [<Element 'nonterminal' at 0x7f14d808fc78>, <Element 'children' at 0x7f14d820bb38>]
__head_unknown__
1
<Element 'node' at 0x7f14d820bf98> {'nid': '1', 'from': '0', 'to': '1', 'subtrees': '1', 'chosen': 'true'} [<Element 'nonterminal' at 0x7f14d808fdb8>, <Element 'children' at 0x7f14d820bbd8>]
-
2
-
3
<Element 'node' at 0x7f14d820bc28> {'nid': '3', 'from': '1', 'to': '3', 'subtrees': '1', 'chosen': 'false'} [<Element 'nonterminal' at 0x7f14d808fc78>, <Element 'children' at 0x7f14d820bb38>]
__head_unknown__
4
<Element 'node' at 0x7f14d820b9f8> {'nid': '4', 'from': '1', 'to': '2', 'subtrees': '1', 'chosen': 'true'} [<Element 'nonterminal' at 0x7f14d8335228>, <Element 'children' at 0x7f14d820ba

In [161]:
ET.dump(random_tree)

<tree grammar_no="1505562921" sent_id="NKJP_1M_2002000170/morph_7-p/morph_7.78-s"><text>- Ani trochę.</text>
  <startnode from="0" to="4">wypowiedzenie</startnode>
  <forest-stats cputime="0.007831746" inferences="12165" nodes="17" trees="3" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="agataw">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="annaa">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <tree-stats height="6" nodes="13" /><node chosen="true" from="0" nid="0" subtrees="3" to="4"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w2"><child from="0" head="false" nid="1" to="1" /><child from="1" head="true" nid="15" to="3" /><child from="3" head="false" nid="11" to="4" /></children></node><node chosen="true" 

In [101]:
def dependency_tree(tree):

    dep_tree, ids = terminals(tree)
    n_terminals = len(dep_tree)
    
    for nid in ids:

        parent = tree.find(".//children/child[@nid='"+str(nid)+"']....")

        if parent is not None:
            loc =  np.where([str(nid) in [x[0] for x in branch] and len(branch[-1])>=2 for branch in dep_tree])[0]

            if parent.attrib["nid"] not in ids:
                ids.append(parent.attrib["nid"])


            if len(parent.findall("children/child"))==1:
                
                dep_tree[loc[0]].append(tuple([parent.attrib["nid"]] +[x.text for x in parent.find("nonterminal").getchildren()]))

                if parent.attrib["nid"] == "0":
                    
                    if parent.attrib["nid"] not in [branch[0][0] for branch in dep_tree]:
                        dep_tree.append([tuple([parent.attrib["nid"]] +[x.text for x in parent.find("nonterminal").getchildren()])])
   
            
            else:
                
                dep_tree[loc[0]].append((parent.attrib["nid"],))
                
                if parent.attrib["nid"] not in [branch[0][0] for branch in dep_tree]:
                    dep_tree.append([tuple([parent.attrib["nid"]] +[x.text for x in parent.find("nonterminal").getchildren()])])
        else:
            pass
            #labels.append(get_subtree_label(tree, tree.find(".//node[@nid='" + str(nid) + "']")))
            
    heads = [x[-2][0] if len(x)>1 else x[0][0]  for x in dep_tree]       
    labels = [get_subtree_label(tree, tree.find(".//node[@nid='" + str(nid) + "']")) for nid in heads]

    return(dep_tree, labels, n_terminals)                      

In [102]:
dep_tree, labels, n = dependency_tree(random_tree)
dep_tree

[[('6', 'Papież', 'papież', 'subst:sg:nom:m1'),
  ('5', 'formarzecz', 'papież', 'nom', 'mos', 'poj', '[]'),
  ('4',
   'fno',
   'papież',
   'nom',
   'mos',
   'poj',
   '3',
   '[]',
   'rzecz',
   'bzap',
   'pre',
   'tak',
   'neut',
   'ni'),
  ('3',)],
 [('9', 'Formosus', 'Formosus', 'subst:sg:nom:m1'),
  ('8', 'formarzecz', 'Formosus', 'nom', 'mos', 'poj', '[]'),
  ('7',
   'fno',
   'Formosus',
   'nom',
   'mos',
   'poj',
   '3',
   '[]',
   'rzecz',
   'bzap',
   'pre',
   'tak',
   'neut',
   'ni'),
  ('3',)],
 [('13', 'zmarł', 'zemrzeć', 'praet:sg:m1:perf'),
  ('12',
   'formaczas',
   'os',
   'dk',
   'prze',
   'ozn',
   'mos',
   'poj',
   '3',
   '[subj(np(nom))]',
   'tak'),
  ('11',
   'fwe',
   'os',
   'dk',
   'prze',
   'ozn',
   'mos',
   'poj',
   '3',
   '[subj(np(nom))]',
   'tak',
   'neut',
   'ni'),
  ('10',
   'ff',
   'os',
   'dk',
   'prze',
   'ozn',
   'mos',
   'poj',
   '3',
   '[subj(np(nom))]',
   'tak',
   'neut',
   'ni'),
  ('1',)],
 [('17'

In [112]:
def transform_to_dependency_format(tree):
    
    dep_tree, labels, n_terminals = dependency_tree(tree)
    
    values = [[x[1] for x in branch[:-1]] for branch in dep_tree]
    
    top_node_ids = [branch[-1][0] for branch in dep_tree]
    
    for i in range(n_terminals,len(values)):
        if len(values[i])>0:
            values[i] = [get_head(tree, top_node_ids[i])]+values[i]
        else:
            values[i] = [get_head(tree, top_node_ids[i]),"__wypowiedzenie__"]


    tokens_and_rules = [(y[0],"-".join(y[1:])) if len(y)>1 else (y[0],"__brak__") for y in values]

    nodes_ids = [[x[0] for x in branch] for branch in dep_tree]
    
    parent_ids = [0]*len(nodes_ids)
    firsts = [x[0] for x in nodes_ids]


    for i in range(len(nodes_ids)):
        last = nodes_ids[i][-1]

        if len(nodes_ids[i])==1 and last == "0":
            parent_ids[i] = 0
        else:
            parent_ids[i] = np.where([last == x for x in firsts])[0][0] + 1 # "+1" po to zeby format danych zgadzal sie z tymi ze stanfordu 
                                                                            # - numerujemy tokeny od 1, a nie od 0

    nodes_used_in_tree = [x[0] for branch in dep_tree for x in branch]    
    
    dependency_data = list(zip([x[0] for x in tokens_and_rules],[x[1] for x in tokens_and_rules], parent_ids, labels))
    
    return(dependency_data, nodes_used_in_tree)


In [150]:
random_tree = get_random_tree(tree)
dep_tree = transform_to_dependency_format(random_tree)
dep_tree[0]

[('Papież', 'formarzecz-fno', 18, 1),
 ('Formosus', 'formarzecz-fno', 18, 1),
 ('zmarł', 'formaczas-fwe-ff', 22, 1),
 ('w', 'przyimek', 14, 1),
 ('kwietniu', 'formarzecz-fno', 19, 1),
 ('896', 'formaprzym-fpt', 20, 1),
 ('w', 'przyimek', 15, 1),
 ('wyniku', 'formarzecz-fno', 21, 0),
 ('nieznanej', 'formaprzym-fpt-fno', 21, 0),
 ('choroby', 'formarzecz-fno', 21, 0),
 ('lub', 'spójnik', 16, 1),
 ('otrucia', 'formarzecz-fno', 16, 1),
 ('.', 'znakkonca', 17, 1),
 ('zmarł', 'fpm-fl', 22, 0),
 ('896', 'fpm', 20, 0),
 ('kwietniu', 'fno', 19, 0),
 ('zmarł', '__wypowiedzenie__', 0, 0),
 ('zmarł', 'fno-fw', 22, 1),
 ('w', 'fno', 14, 0),
 ('lub', 'fpt-fno', 16, 0),
 ('w', 'fno', 15, 0),
 ('zmarł', 'zdanie', 17, 0)]

In [135]:
ET.dump(trees[0])

NameError: name 'trees' is not defined

In [120]:
def write_dependency_format(dep_tree, folder, overwrite=False):
    
    if not overwrite:
        mode = "a+"
    else:
        mode = "w"
    
    tokens = [x[0] for x in dep_tree[0]]
    with open(folder+"/tokens.txt", mode) as f:
        f.write(" ".join(tokens) + "\n")
        
    rules = [x[1] for x in dep_tree[0]]
    with open(folder+"/rules.txt", mode) as f:
        f.write(" ".join(rules) + "\n")
        
    parents = [str(x[2]) for x in dep_tree[0]]
    with open(folder+"/parents.txt", mode) as f:
        f.write(" ".join(parents) + "\n")
        
    labels = [str(x[3]) for x in dep_tree[0]]
    with open(folder+"/labels.txt", mode) as f:
        f.write(" ".join(labels) + "\n")

    nodes_used_in_tree = dep_tree[1]
    with open(folder+"/nodes_used_in_tree.txt", mode) as f:
        f.write(" ".join(nodes_used_in_tree) + "\n")

In [121]:
write_dependency_format(dep_tree, "Data")

In [278]:
data_folder = "../Składnica-frazowa-171220/**/*.xml"

In [362]:
import glob

j = 1
trees, labels = [],[]
for filename in glob.iglob(data_folder, recursive=True):
    
    forest = ET.parse(filename)
 
    try:
        num_trees = number_of_trees_in_forest(forest)
        
        
        if num_trees<100000:
            
            if j % 100 ==0:
                print(j)
                print(num_trees)
            
            trees.append(get_positive_tree(forest))
            labels.append(1)
        
            
            if num_trees < 10:

                trees.append(get_random_negative_tree(forest))
                labels.append(0)

            elif num_trees<20:
                for i in range(3):
                    trees.append(get_random_negative_tree(forest))
                    labels.append(0)
            elif num_trees<30:
                for i in range(4):
                    trees.append(get_random_negative_tree(forest))
                    labels.append(0)
            elif num_trees<40:
                for i in range(5):
                    trees.append(get_random_negative_tree(forest))
                    labels.append(0)
            elif num_trees<10000:
                for i in range(10):
                    trees.append(get_random_negative_tree(forest))
                    labels.append(0)
               
            j += 1
        
    except:
        pass
    
    

    if j>1000:
        break

100
1
100
10


KeyboardInterrupt: 

In [12]:
len(trees)

NameError: name 'trees' is not defined

In [364]:
for i, tree in enumerate(trees):
    if i%50 ==0:
        print(i)
    write_dependency_format(transform_to_dependency_format(tree), "Data")

0
50


KeyError: 'chosen'

In [368]:
tree.write("test2.xml")

In [9]:
def _tree_height(xml_tree, node_id=0):
    
    """
    Funkcja oblicza wysokosc drzewa (dlugosc najdluzszej sciezki od korzenia do liscia)
    lub lasu (maximum z wszystkich mozliwych drzew)
    
    xml_tree - drzewo luba las drzew lub korzen drzewa jednego lub drugiego
    """
    
    
    if type(xml_tree)==ET.Element:
        node = tree
    else:
        node = tree.getroot()
        
    node = node.find('.//node[@nid="' + str(node_id) + '"]')
    children = node.findall(".//children//child")
    
    if len(children)==0:
        return 1
    else:
        children_nodes = [child.attrib["nid"] for child in children]
        return 1+max([_tree_height(tree,x) for x in children_nodes])
        

In [10]:
_tree_height(tree.getroot())

15

In [18]:
def number_of_nodes(tree):
    """
    Zwraca liczbe wezlow w drzewie.
    
    tree - drzewo lub korzen drzewa
    """
    if type(tree)==ET.Element:
        return len(tree.findall("node"))
    else:
        return len(tree.getroot().findall("node")) 

In [15]:
number_of_nodes(tree)

124

In [23]:
def get_subtree_label(tree, node):
    
    if node.find("children") is None:
        return 1
    
    if node.find("children").attrib.get("chosen","false") == "false":
        return 0
    else:
        return int(np.all([get_subtree_label(tree, tree.find(".//node[@nid='"+ x.attrib["nid"] + "']")) for x in node.find("children").findall("child")]))

In [20]:
def is_positive(tree): 
    
    """
    Funkcja sprawdza czy drzewo jest pozytywne - czy jest poprawnym drzewem rozbioru
    Zwraca wartosc logiczna.
    
    tree - drzewo [xml.etree.ElementTree.ElementTree]
    """
    
    _check_sentence(tree,"tree")
    
    assert len(tree.find("node"))>0, 'There is not "node" element in the tree'
    
    #Sprawdzamy czy wszystkie wezly "node" maja wartosc chosen="true":
    for x in tree.iter("node"):
        if not x.attrib["chosen"]=="true":
            return False

    #Sprawdzamy czy wszystkie wezly "children" maja wartosc chosen="true":
    for x in tree.iter(".//children"):
        if not x.attrib["chosen"]=="true":
            return False
        
    return True
        
        
    


def get_positive_tree(forest):
    
    """
    Funkcja zwraca poprawne (pozytywne) drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie komunikatu
    _check_sentence(forest,"forest")
            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    def add_positive_children(current_node_old):
        
        current_node_new = ET.SubElement(root_new, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        
        children_old = current_node_old.findall('children[@chosen="true"]')
        # powinno byc tylko jedno takie dziecko
        
        assert len(children_old) <= 1, 'More than one children has chosen="true"'
        
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return None
        
        #random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        children_new = ET.SubElement(current_node_new, children_old[0].tag, children_old[0].attrib)
        for child_old in children_old[0].getchildren():
            x = ET.SubElement(children_new, child_old.tag, child_old.attrib)
            next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
            assert next_node.attrib["chosen"] == "true"
            add_positive_children(next_node)
        
    
    # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    # TODO: upewnic sie czy to jest poprawne podejscie - czy moze byc inny wezel poczatkowym
    node_0 = root_old.find('.//node[@nid="0"][@chosen="true"]') 
    
    # konstruujemy drzewo:
    add_positive_children(node_0)
    
    positive_tree = ET.ElementTree(root_new)

    # Sprawdzenie poprawnosci drzewa
    assert is_positive(positive_tree), """Something gone wrong - tree is not positive"""
        
        
    return positive_tree
    

In [10]:
pos_tree = get_positive_tree(tree)

In [83]:
get_positive_tree(tree).write("test.xml")

In [96]:
def get_all_trees(forest):
    
    """
    Funkcja zwraca losowe drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie komunikatu
    _check_sentence(forest,"forest")

            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
    
    # definiujemy wezel ze statystykami drzewa
    # robimy to w tym iejscu zeby zachowac logiczna kolejnosc wezlow - zeby wypisywalo sie to na poczatku
    # wartosci nadamy nizej
    ET.SubElement(root_new, "tree-stats", {"height":"0","nodes":"0"})
            
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    
    trees = []
    
    def add_random_children(current_node_old,root_new):
        
        root_new_recurrent = deepcopy(root_new)
        
        current_node_new = ET.SubElement(root_new_recurrent, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        children_old = current_node_old.findall("children")
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return root_new_recurrent
        
        #random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        #random_children_new = ET.SubElement(current_node_new, random_children_old.tag, random_children_old.attrib)
        
        
        
        for children in children_old:
            
            children_new = ET.SubElement(current_node_new, children.tag, children.attrib)
            
            for child_old in children.getchildren():
                x = ET.SubElement(children_new, child_old.tag, child_old.attrib)
                next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
                if len(next_node.findall("children"))==0:
                    trees.append(ET.ElementTree(root_new_recurrent))
                else:
                    add_random_children(next_node,root_new_recurrent)
                    
        
        
    
        # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    # TODO: upewnic sie czy to jest poprawne podejscie - czy moze byc inny wezel poczatkowym
    node_0 = root_old.find('.//node[@nid="0"]') 
    
    # konstruujemy drzewo:
    #add_random_children(node_0,root_new)
    
    #new_tree = ET.ElementTree(root_new)
    
    #th = _tree_height(new_tree, node_id=0)
    
    #root_new.find("tree-stats").attrib["height"] = str(th)
    #root_new.find("tree-stats").attrib["nodes"] = str(len(root_new.findall("node")))
    
    add_random_children(node_0,root_new)
    
    return trees

In [97]:
d = get_all_trees(tree)

In [94]:
ET.dump(d[0])

<tree grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest-stats cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <tree-stats height="0" nodes="0" /><node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /></c

In [99]:
number_of_trees_in_forest(tree)

173

In [98]:
len(d)

804

In [105]:
d[0].getroot().getchildren()

[<Element 'text' at 0x7fb0a63eb818>,
 <Element 'startnode' at 0x7fb0a63eb868>,
 <Element 'forest-stats' at 0x7fb0a63eb8b8>,
 <Element 'answer-data' at 0x7fb0a63eb908>,
 <Element 'tree-stats' at 0x7fb0a63ebb38>,
 <Element 'node' at 0x7fb0a63ebb88>,
 <Element 'node' at 0x7fb0a63ebd18>,
 <Element 'node' at 0x7fb0a63e9228>,
 <Element 'node' at 0x7fb0a63e9728>,
 <Element 'node' at 0x7fb0a63e9c78>,
 <Element 'node' at 0x7fb0a63df228>]

In [106]:
ET.dump(d[0])

<tree grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest-stats cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <tree-stats height="0" nodes="0" /><node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /></children></node><node chosen="true" from="0" nid="

'c.xml'

In [93]:
positive_tree = get_positive_tree(tree)

In [125]:
new_tree = get_random_negative_tree(tree)

In [127]:
scores(positive_tree, new_tree)

(0.851063829787234, 0.8163265306122449, 0.8333333333333334)

In [129]:
f1=[]
for i in range(100):
    new_tree = get_random_negative_tree(tree)
    f1.append(scores(positive_tree, new_tree)[2])

In [132]:
np.mean(f1)

0.819978025574491

In [ ]:
len()